### Постановка задачи
Заполним отсутствующие значения по погоде интерполяционными данными.

Для точки росы вычтем температуру воздуха, направление ветра разложим на синус и косинус, для температуры воздуха вычислим  первую и вторую производные. Также введем параметры по праздничным дням, дням недели, месяцам и неделям года.

Посчитаем модель линейной регрессии по первым 20 зданиям и найдем ее точность. Проверим, какой набор параметров позволяет улучшить точность.

Данные:
* http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz
Соревнование: https://www.kaggle.com/c/ashrae-energy-prediction/

© ITtensive, 2020

### Подключение библиотек

In [1]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import numpy as np
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

### Загрузка данных, отсечение 20 зданий, объединение и оптимизация

In [2]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif col == "timestamp":
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != "datetime":
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [3]:
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
weather = weather[weather["site_id"] == 0]
energy = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz")
energy = energy[energy["building_id"]<20]
energy = pd.merge(left=energy, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
del buildings
print (energy.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175680 entries, 0 to 175679
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   building_id    175680 non-null  int64  
 1   meter          175680 non-null  int64  
 2   timestamp      175680 non-null  object 
 3   meter_reading  175680 non-null  float64
 4   site_id        175680 non-null  int64  
 5   primary_use    175680 non-null  object 
 6   square_feet    175680 non-null  int64  
 7   year_built     175680 non-null  float64
 8   floor_count    0 non-null       float64
dtypes: float64(3), int64(4), object(2)
memory usage: 13.4+ MB
None


### Интерполяция значений

In [4]:
weather["precip_depth_1_hr"] = weather["precip_depth_1_hr"].apply(lambda x:x if x>0 else 0)
interpolate_columns = ["air_temperature", "dew_temperature",
                       "cloud_coverage", "wind_speed", "wind_direction",
                       "precip_depth_1_hr", "sea_level_pressure"]
for col in interpolate_columns:
    weather[col] = weather[col].interpolate(limit_direction='both',
                            kind='cubic')

### Обогащение данных: погода
Направление ветра разложим на синус и косинус, для температуры воздуха вычислим  первую и вторую производные

In [5]:
# Перевод в Пи
weather["wind_direction_rad"] = weather["wind_direction"] * np.pi/180
weather["wind_direction_sin"] = np.sin(weather["wind_direction_rad"])
weather["wind_direction_cos"] = np.cos(weather["wind_direction_rad"])
# Вычисление производных. Продублируем первое значение, чтобы было как считать диффы. 
weather["air_temperature_diff1"] = weather["air_temperature"].diff()
weather.at[0, "air_temperature_diff1"] = weather.at[1, "air_temperature_diff1"]
weather["air_temperature_diff2"] = weather["air_temperature_diff1"].diff()
weather.at[0, "air_temperature_diff2"] = weather.at[1, "air_temperature_diff2"]

### Объдинение погодных данных

In [6]:
energy = energy.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])
energy = pd.merge(left=energy, right=weather, how="left",
                  left_index=True, right_index=True)
energy.reset_index(inplace=True)
energy = energy.drop(columns=["meter", "site_id", "year_built",
                              "square_feet", "floor_count"], axis=1)
del weather
energy = reduce_mem_usage(energy)
print (energy.info())

Потребление памяти меньше на 15.41 Мб (минус 71.9 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175680 entries, 0 to 175679
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   timestamp              175680 non-null  datetime64[ns]
 1   building_id            175680 non-null  int8          
 2   meter_reading          175680 non-null  float16       
 3   primary_use            175680 non-null  category      
 4   air_temperature        175680 non-null  float16       
 5   cloud_coverage         175680 non-null  float16       
 6   dew_temperature        175680 non-null  float16       
 7   precip_depth_1_hr      175680 non-null  float16       
 8   sea_level_pressure     175680 non-null  float16       
 9   wind_direction         175680 non-null  float16       
 10  wind_speed             175680 non-null  float16       
 11  wind_direction_rad     175680 non-null  float16    

### Обогащение данных: дата

In [7]:
energy["hour"] = energy["timestamp"].dt.hour.astype("int8")
energy["weekday"] = energy["timestamp"].dt.weekday.astype("int8")
energy["week"] = energy["timestamp"].dt.week.astype("int8")
energy["month"] = energy["timestamp"].dt.month.astype("int8")
energy["date"] = pd.to_datetime(energy["timestamp"].dt.date)
dates_range = pd.date_range(start='2015-12-31', end='2017-01-01')
us_holidays = calendar().holidays(start=dates_range.min(),
                                  end=dates_range.max())
energy['is_holiday'] = energy['date'].isin(us_holidays).astype("int8")
for weekday in range(0,7):
    energy['is_wday' + str(weekday)] = energy['weekday'].isin([weekday]).astype("int8")
for week in range(1,54):
    energy['is_w' + str(week)] = energy['week'].isin([week]).astype("int8")
for month in range(1,13):
    energy['is_m' + str(month)] = energy['month'].isin([month]).astype("int8")

<ipython-input-7-63a20a98c042>:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  energy["week"] = energy["timestamp"].dt.week.astype("int8")


### Логарифмирование данных
z = A * x + B * y -> log z = A * x + B * y => z = e^Ax * e^By => z = a^x * b^y

In [8]:
energy["meter_reading_log"] = np.log(energy["meter_reading"] + 1)

### Разделение данных

In [9]:
energy_train, energy_test = train_test_split(energy[energy["meter_reading"]>0], test_size=0.2)
print (energy_train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86858 entries, 143057 to 150137
Data columns (total 95 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   timestamp              86858 non-null  datetime64[ns]
 1   building_id            86858 non-null  int8          
 2   meter_reading          86858 non-null  float16       
 3   primary_use            86858 non-null  category      
 4   air_temperature        86858 non-null  float16       
 5   cloud_coverage         86858 non-null  float16       
 6   dew_temperature        86858 non-null  float16       
 7   precip_depth_1_hr      86858 non-null  float16       
 8   sea_level_pressure     86858 non-null  float16       
 9   wind_direction         86858 non-null  float16       
 10  wind_speed             86858 non-null  float16       
 11  wind_direction_rad     86858 non-null  float16       
 12  wind_direction_sin     86858 non-null  float16       


### Линейная регрессия

In [10]:
hours = range(0, 24)
buildings = range(0, energy_train["building_id"].max() + 1)
lr_columns = ["meter_reading_log", "hour", "building_id",
             "air_temperature", "dew_temperature",
             "sea_level_pressure", "wind_speed", "cloud_coverage",
             "air_temperature_diff1", "air_temperature_diff2",
             "is_holiday"]
for wday in range(0,7):
    lr_columns.append("is_wday" + str(wday))
for week in range(1,54):
    lr_columns.append("is_w" + str(week))
for month in range(1,13):
    lr_columns.append("is_m" + str(month))
lr_columns

['meter_reading_log',
 'hour',
 'building_id',
 'air_temperature',
 'dew_temperature',
 'sea_level_pressure',
 'wind_speed',
 'cloud_coverage',
 'air_temperature_diff1',
 'air_temperature_diff2',
 'is_holiday',
 'is_wday0',
 'is_wday1',
 'is_wday2',
 'is_wday3',
 'is_wday4',
 'is_wday5',
 'is_wday6',
 'is_w1',
 'is_w2',
 'is_w3',
 'is_w4',
 'is_w5',
 'is_w6',
 'is_w7',
 'is_w8',
 'is_w9',
 'is_w10',
 'is_w11',
 'is_w12',
 'is_w13',
 'is_w14',
 'is_w15',
 'is_w16',
 'is_w17',
 'is_w18',
 'is_w19',
 'is_w20',
 'is_w21',
 'is_w22',
 'is_w23',
 'is_w24',
 'is_w25',
 'is_w26',
 'is_w27',
 'is_w28',
 'is_w29',
 'is_w30',
 'is_w31',
 'is_w32',
 'is_w33',
 'is_w34',
 'is_w35',
 'is_w36',
 'is_w37',
 'is_w38',
 'is_w39',
 'is_w40',
 'is_w41',
 'is_w42',
 'is_w43',
 'is_w44',
 'is_w45',
 'is_w46',
 'is_w47',
 'is_w48',
 'is_w49',
 'is_w50',
 'is_w51',
 'is_w52',
 'is_w53',
 'is_m1',
 'is_m2',
 'is_m3',
 'is_m4',
 'is_m5',
 'is_m6',
 'is_m7',
 'is_m8',
 'is_m9',
 'is_m10',
 'is_m11',
 'is_m12']

In [11]:
energy_train_lr = pd.DataFrame(energy_train, columns=lr_columns)
energy_train_lr.head()

,meter_reading_log,hour,building_id,air_temperature,dew_temperature,sea_level_pressure,wind_speed,cloud_coverage,air_temperature_diff1,air_temperature_diff2,...,is_m3,is_m4,is_m5,is_m6,is_m7,is_m8,is_m9,is_m10,is_m11,is_m12
143057,4.777344,0,17,20.593750,13.898438,1022.0,2.599609,0.000000,-1.099609,1.599609,...,0,0,0,0,0,0,0,1,0,0
144596,7.144531,5,16,22.203125,17.796875,1021.0,3.599609,8.000000,0.000000,0.600098,...,0,0,0,0,0,0,0,1,0,0
68610,7.078125,22,10,30.000000,13.296875,1011.0,7.699219,5.000000,-1.099609,-0.500000,...,0,0,1,0,0,0,0,0,0,0
101535,5.519531,12,15,26.093750,24.406250,1017.5,0.000000,2.000000,1.700195,1.700195,...,0,0,0,0,1,0,0,0,0,0
91707,6.199219,1,7,27.796875,22.796875,1017.5,3.099609,6.332031,-0.500000,4.000000,...,0,0,0,0,1,0,0,0,0,0


In [13]:
energy_lr = [[]]*len(buildings)
energy_lr

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [14]:

for building in buildings:
    energy_lr[building] = [[]]*len(hours)
    energy_train_b = energy_train_lr[energy_train_lr["building_id"]==building]
    for hour in hours:
        energy_train_bh = energy_train_b[energy_train_b["hour"]==hour]
        y = energy_train_bh["meter_reading_log"]
        x = energy_train_bh.drop(labels=["meter_reading_log",
            "hour", "building_id"], axis=1)
        model = LinearRegression(fit_intercept=False).fit(x, y)
        energy_lr[building][hour] = model.coef_
        energy_lr[building][hour] = np.append(energy_lr[building][hour], model.intercept_)


In [16]:
energy_lr

[[array([ 2.15504249e-03,  9.88762639e-03, -6.62812218e-03,  4.99313697e-03,
         -1.13488138e-02,  7.18135387e-03, -1.08042099e-02, -1.33172587e-01,
          5.77891397e+00,  5.74769735e+00,  5.76129103e+00,  5.74490356e+00,
          5.74988079e+00,  5.71765709e+00,  5.73079395e+00, -1.02251768e-04,
         -1.04546547e-04,  8.60691071e-05, -4.29153442e-06,  4.13656235e-05,
         -2.71797180e-05, -1.33156776e-04,  2.59876251e-05,  2.07424164e-05,
          2.31266022e-05,  1.26361847e-05, -9.42945480e-05, -1.07288361e-06,
          1.57356262e-05, -3.81469727e-06, -7.86781311e-06,  8.58306885e-06,
          9.53674316e-07, -2.38418579e-06,  1.49717772e+00,  1.30312586e+00,
          1.21810985e+00,  1.29785812e+00,  1.44528031e+00,  1.30241561e+00,
          1.25376058e+00,  1.24498367e+00,  1.44804335e+00,  1.42233825e+00,
          1.36596715e+00,  1.33937657e+00,  1.35556257e+00,  1.43382823e+00,
          1.26589823e+00,  1.19569540e+00,  1.21868908e+00,  1.18868518e+00,

In [17]:
energy_lr[0]

[array([ 2.15504249e-03,  9.88762639e-03, -6.62812218e-03,  4.99313697e-03,
        -1.13488138e-02,  7.18135387e-03, -1.08042099e-02, -1.33172587e-01,
         5.77891397e+00,  5.74769735e+00,  5.76129103e+00,  5.74490356e+00,
         5.74988079e+00,  5.71765709e+00,  5.73079395e+00, -1.02251768e-04,
        -1.04546547e-04,  8.60691071e-05, -4.29153442e-06,  4.13656235e-05,
        -2.71797180e-05, -1.33156776e-04,  2.59876251e-05,  2.07424164e-05,
         2.31266022e-05,  1.26361847e-05, -9.42945480e-05, -1.07288361e-06,
         1.57356262e-05, -3.81469727e-06, -7.86781311e-06,  8.58306885e-06,
         9.53674316e-07, -2.38418579e-06,  1.49717772e+00,  1.30312586e+00,
         1.21810985e+00,  1.29785812e+00,  1.44528031e+00,  1.30241561e+00,
         1.25376058e+00,  1.24498367e+00,  1.44804335e+00,  1.42233825e+00,
         1.36596715e+00,  1.33937657e+00,  1.35556257e+00,  1.43382823e+00,
         1.26589823e+00,  1.19569540e+00,  1.21868908e+00,  1.18868518e+00,
         1.1

### Расчет качества

In [18]:
def calculate_model (x):
    lr = -1
    model = energy_lr[x.building_id][x.hour]
    if len(model) > 0:
        lr = np.sum([x[col] * model[i] for i,col in enumerate(lr_columns[3:])])
        lr += model[len(lr_columns)-3]
        lr = np.exp(lr)
    if lr < 0:
        lr = 0
    x["meter_reading_lr_q"] = (np.log(x.meter_reading + 1) -
                               np.log(1 + lr))**2
    return x


In [19]:
energy_test.head()

,timestamp,building_id,meter_reading,primary_use,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,...,is_m4,is_m5,is_m6,is_m7,is_m8,is_m9,is_m10,is_m11,is_m12,meter_reading_log
154849,2016-11-18 14:00:00,9,186.3750,Office,20.593750,2.000000,13.296875,0.0,1022.5,360.0,...,0,0,0,0,0,0,0,1,0,5.234375
147209,2016-11-02 16:00:00,9,171.1250,Office,27.796875,6.000000,18.906250,0.0,1023.0,70.0,...,0,0,0,0,0,0,0,1,0,5.148438
72428,2016-05-30 21:00:00,8,354.2500,Education,28.296875,4.800781,18.906250,0.0,1013.0,80.0,...,0,1,0,0,0,0,0,0,0,5.871094
162701,2016-12-04 23:00:00,1,95.8125,Education,23.906250,2.000000,16.703125,0.0,1016.0,130.0,...,0,0,0,0,0,0,0,0,1,4.574219
171908,2016-12-24 03:00:00,8,477.0000,Education,20.000000,0.000000,16.703125,0.0,1025.0,60.0,...,0,0,0,0,0,0,0,0,1,6.167969


In [20]:

energy_test = energy_test.apply(calculate_model,
                                    axis=1, result_type="expand")
energy_test.head()


,timestamp,building_id,meter_reading,primary_use,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,...,is_m5,is_m6,is_m7,is_m8,is_m9,is_m10,is_m11,is_m12,meter_reading_log,meter_reading_lr_q
154849,2016-11-18 14:00:00,9,186.3750,Office,20.593750,2.000000,13.296875,0.0,1022.5,360.0,...,0,0,0,0,0,0,1,0,5.234375,0.041749
147209,2016-11-02 16:00:00,9,171.1250,Office,27.796875,6.000000,18.906250,0.0,1023.0,70.0,...,0,0,0,0,0,0,1,0,5.148438,0.003948
72428,2016-05-30 21:00:00,8,354.2500,Education,28.296875,4.800781,18.906250,0.0,1013.0,80.0,...,1,0,0,0,0,0,0,0,5.871094,0.003459
162701,2016-12-04 23:00:00,1,95.8125,Education,23.906250,2.000000,16.703125,0.0,1016.0,130.0,...,0,0,0,0,0,0,0,1,4.574219,0.026873
171908,2016-12-24 03:00:00,8,477.0000,Education,20.000000,0.000000,16.703125,0.0,1025.0,60.0,...,0,0,0,0,0,0,0,1,6.167969,0.000815


In [21]:
energy_test_lr_rmsle = np.sqrt(energy_test["meter_reading_lr_q"].sum() / len(energy_test))
print ("Качество линейной регрессии, 20 зданий:",
       energy_test_lr_rmsle, round(energy_test_lr_rmsle, 1))

Качество линейной регрессии, 20 зданий: 0.221964415583969 0.2
